Questa versione della rumor classification usa per il raining un dataset bilanciato, ovvero solo i rumor etichettati del training test
Essendo pochi (una novantina) come classificatore si usa sempre il NaiveBayes

Apertura del file di training

In [2]:
import json

with open('English_train_v2.json', 'r') as f:
    trainSet = json.load(f)


Apertura del file di test

In [3]:
with open('English_test_v2.json', 'r') as f:
    testSet = json.load(f)

Vettorizzazione del training set

In [5]:
#Lista di tutti i rumor
listaRumor = []
for oggetto in trainSet:
    listaRumor.append(oggetto["rumor"])

Abbiamo 96 eempi di training

In [23]:
dimTrain = len(listaRumor)

Faccio la stessa cosa per il test set

In [8]:
#Lista di tutti i rumor
listaRumorTest = []
for oggetto in testSet:
    listaRumorTest.append(oggetto["rumor"])

Abbiamo 32 esempi di test

In [24]:
dimTest = len(listaRumorTest)

Creazione di una lista unica da vettorizzare

In [11]:
listaDaVett = listaRumor + listaRumorTest

In [12]:
len(listaDaVett)

128

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import heapq
import re

def flat_list(lista):
    ris = []
    for elemento in lista.toarray():
        ris.extend(elemento)
    return ris

def vettorizzazione(rumors):
    pattern = r"\bhttps?.*\b"
    patternHashtag = r"\b\#(.*)\b"
    tweet = []
    for element in rumors:
        tweet.append(re.sub(pattern, '', element))

    #Calcolo coseno
    vectorizer = TfidfVectorizer(use_idf=True,sublinear_tf=True,smooth_idf=True, stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(tweet)

    return tfidf_matrix

Vettorizzo

In [21]:
listaVett = vettorizzazione(listaDaVett)

In [22]:
listaVett

<128x1442 sparse matrix of type '<class 'numpy.float64'>'
	with 2987 stored elements in Compressed Sparse Row format>

Separo le vettorizzazioni di training e edi test

In [25]:
trainVett = listaVett[:-dimTest]
testVett = listaVett[-dimTest:]

In [27]:
X = trainVett

Creo le label di test

In [28]:
Y = []
for oggetto in trainSet:
    Y.append(oggetto["label"])

In [30]:
len(Y)

96

Addestramento del modello

In [31]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()

model= NB.fit(X,Y)

Adesso vettorizzazione del test e previsione

Devo estrarre quelli più significativi e vettorizzarli

In [33]:
ris = model.predict(testVett)

In [35]:
len(ris)

32

In [38]:
print(ris)

['NOT ENOUGH INFO' 'REFUTES' 'REFUTES' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO'
 'REFUTES' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'REFUTES'
 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO' 'REFUTES' 'NOT ENOUGH INFO' 'REFUTES' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO' 'REFUTES' 'NOT ENOUGH INFO' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO']


Apro il file con le etichette e verifico l'accuratezza

In [39]:
with open('Validation_set.json', 'r') as f:
    validationSet = json.load(f)

predicted_labels = []
for element in validationSet:
    predicted_labels.append(element['predicted_label'])

In [41]:
len(predicted_labels)

32

Verifico l'accuratezza del modello

In [42]:
model.score(testVett,predicted_labels)

0.625